In [1]:
import os
import numpy as np
from numpy import random

from random import sample

import pandas as pd 

import skimage
from skimage import io
from skimage.feature import daisy, hog, ORB, local_binary_pattern, SIFT
from skimage.color import label2rgb, rgb2gray
from skimage.transform import resize, rotate, downscale_local_mean

from scipy import ndimage as ndi
from skimage.util import img_as_float
from skimage.filters import gabor_kernel

from sklearn.cluster import MiniBatchKMeans
from sklearn.decomposition import PCA
from sklearn import preprocessing
from skimage import exposure

from sklearn.preprocessing import Normalizer
from matplotlib.ticker import MaxNLocator
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt

import gc

from joblib import Parallel, delayed, parallel_backend
import psutil

from platform import python_version


import gabor_filters
from  gabor_filters import gabor_filter
from  gabor_filters import gabor_filter_response

import importlib
importlib.reload(gabor_filters)
importlib.reload(gabor_filters.gabor_filter)
importlib.reload(gabor_filters.gabor_filter_response)

from gabor_filters.gabor_filter import GaborFilterBank as gbb
from gabor_filters.gabor_filter_response import GaborFilteredResponseBank as gbfrb

In [2]:
print(python_version())
print(skimage.__version__)

3.8.16
0.19.2


In [3]:
from tqdm.notebook import tqdm

In [4]:
def extract_gabor_from_filepath(filepath):
    # read image from its path
    img = io.imread(filepath, as_gray=True)
    
    # Create Gabor filter bank
    fmax = 0.327 # maximum frequency
    k = np.sqrt(2) #frequency ratio or factor for selecting filter frequencies
    p = 0.5 # crossing point between two consecutive filters, default 0.5
    u = 6 #number of frequencies
    v = 8 #number of orientation
    gamma = 0.5  #smoothting parameter 
    eta = 0.5  #smoothting parameter of
    row = img.shape[0]
    col = img.shape[1] # size of image

    GaborFilterBank = gbb().create_a_set_of_gabor_filters(fmax, k, p, u, v, row, col, gamma, eta)
    
    # Filter with the filter bank
    GaborFilteredReponses = gbfrb().create_a_set_of_Gabor_filtered_responses(img, GaborFilterBank)

    # Convert responses to simple 3-D matrix with normalization
    filteredImages = gbfrb().convert_a_set_Gabor_filtered_responses_to_ndarray(GaborFilteredReponses)
    
    # Get mean and standard deviation of each response as Gabor (texture) features of an input image
    nImages = filteredImages.shape[2]
    textureFeatures = np.zeros(nImages*2)

    index=0
    for i in range(0, nImages):
        textureFeatures[index] = np.mean(np.abs(filteredImages[:,:,i]));
        index = index + 1;
        textureFeatures[index] = np.std(np.abs(filteredImages[:,:,i]));
        index = index + 1;
    
    return textureFeatures

def extract_gabor(dfDataset):
    # chunk the large dataset int smaller pieces
    n = 1000
    list_dfDataset_chunk = [dfDataset[i:i+n] for i in range(0, len(dfDataset), n)]
    
    gabor_list = [None]*len(list_dfDataset_chunk)

    with parallel_backend("loky", inner_max_num_threads=2):
        with Parallel(n_jobs=10, require='sharedmem') as parallel:
            for i, dfDataset_chunk in enumerate(tqdm(list_dfDataset_chunk, desc='Processing data', colour='blue', position=0, leave=True)):
            
                gabor_features = parallel(
                                delayed(extract_gabor_from_filepath)(filepath) for filepath in tqdm(dfDataset_chunk['filenames'], desc='Extract Gabor', colour='cyan', position=1, leave=False)
                            )
            
                gabor_list[i] = gabor_features

                del  gabor_features
                gc.collect()
    
    return gabor_list


def extract_gabor_2(dfDataset):
    
    # with parallel_backend("loky", inner_max_num_threads=3):
    with Parallel(n_jobs=30, require='sharedmem', return_generator=True) as parallel:
        
        gabor_features = parallel(
                        delayed(extract_gabor_from_filepath)(filepath) for filepath in tqdm(dfDataset['filenames'], desc='Extract Gabor', colour='cyan')
                    )
            
    return gabor_features

## 4.2. main()

### 4.2.1. For fold 1
#### 1. Read path of fold 1 file

In [5]:
dfFoldTraining_1 = pd.read_csv('..//_inputs//_images_Zooscan//_Zooscan-training-fold_1.csv')
dfFoldValidation_1 = pd.read_csv('..//_inputs//_images_Zooscan//_Zooscan-validation-fold_1.csv')

In [6]:
display(dfFoldTraining_1.head(5), dfFoldTraining_1.shape)

,filenames,labels,short_filenames,cls
0,..//_inputs//_images_Zooscan//_training//aggre...,aggregats_debris,0001-aggregates.png,0
1,..//_inputs//_images_Zooscan//_training//aggre...,aggregats_debris,0002.png,0
2,..//_inputs//_images_Zooscan//_training//aggre...,aggregats_debris,0003-aggregates.png,0
3,..//_inputs//_images_Zooscan//_training//aggre...,aggregats_debris,0004-aggregates.png,0
4,..//_inputs//_images_Zooscan//_training//aggre...,aggregats_debris,0004.png,0


(44099, 4)

#### 2. Extracting gabor feature for the training set

In [6]:
list_gabor_train = extract_gabor_2(dfFoldTraining_1[:100])

Extract Gabor:   0%|          | 0/100 [00:00<?, ?it/s]

In [9]:
list_gabor_train2 = np.vstack(list_gabor_train)

In [11]:
list_gabor_train2.shape

(100, 96)

In [12]:
del list_gabor_train2, list_gabor_train

gc.collect()

471

In [ ]:
n = 2000

list_dfFoldTraining_1_chunked = [dfFoldTraining_1[i:i+n] for i in range(0, len(dfFoldTraining_1), n)]

display(len(list_dfFoldTraining_1_chunked))

In [ ]:
with Parallel(n_jobs=2) as parallel:
    

In [ ]:
gabor_list_train = None

pbar = tqdm(list_dfFoldTraining_1_chunked)

for i, dfFoldTraining_1_chunked in enumerate(pbar):
    pbar.set_description(f'Processing the chunked data {i+1}')
    
    gabor_list_train_chunked = extract_gabor(dfFoldTraining_1_chunked)

    gabor_list_train = np.vstack(gabor_list_train_chunked)

    del  gabor_list_train_chunked
    gc.collect()

In [ ]:
# create a standard deviation normalization for later uses
train_std_norm = StandardScaler().fit(HOG_list)

In [ ]:
# Standard deviation normalization
HOG_list_std = train_std_norm.transform(HOG_list)

In [ ]:
display(HOG_list_std)

In [ ]:
print('Total HOG features:',(HOG_list_std.shape))

#### 3. PCA analysis on the training set

In [ ]:
pca_HOG_std = PCA().fit(HOG_list_std)

##### 3.1. Plot PCA components and CEV

From this, we can know number of components to keep

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5), tight_layout=True)

ax.plot(np.cumsum(pca_HOG_std.explained_variance_ratio_)*100, linewidth=2)
ax.grid(color='r', linestyle='--', linewidth=1)

ax.set_xlabel('Number of components')
ax.set_ylabel('Cumulative explained variance');

ax.set_yticks(np.arange(0,105,5))
ax.set_xticks(np.arange(0,HOG_list.shape[1],100))

ax.axhline(y=90, linewidth=3, color='g', alpha=0.5)

# ax.plot(800, 91, marker="o", markersize=10, markeredgecolor="red", markerfacecolor="green")

ax.set_title("PCA analysis on HOG features of training set")

In [ ]:
(np.cumsum(pca_HOG_std.explained_variance_ratio_)*100)[1100]

<b>Remarks: More than 90% of variance is explained by first 1100 components</b>

##### 3.2. Kaiser's rule in statistics: Pick components which have eigenvalues >= 1 or 0.7


In [ ]:
fig, ax = plt.subplots(figsize=(10, 5), tight_layout=True)

ax.plot(pca_HOG_std.explained_variance_, 'bo-', linewidth=2)
ax.grid(color='r', linestyle='--', linewidth=1)

ax.set_yticks(np.arange(0,125,5))
ax.set_xticks(np.arange(0,HOG_list.shape[1],100))

ax.set_xlabel('Principal Component')
ax.set_ylabel('Eigenvalue')
plt.axhline(y=1, linewidth=1, color='g', alpha=0.5)
plt.title('Scree Plot of PCA: Component Eigenvalues')

In [ ]:
print('\nEigenvalues \n%s' %pca_HOG_std.explained_variance_)
print('Eigenvectors \n%s' %pca_HOG_std.components_)

In [ ]:
# kaiser's rule in statistics: Pick components which have eigenvalues >= 1 or 0.7
a = pca_HOG_std.explained_variance_ >= 1.0

In [ ]:
a.sum()

<b> Only 309 components are significant and should be kept </b>

#### 4. Fit PCA to the HOG features

1. We fit PCA (with n_components to keep) onto the training set
2. Transform the training set with that PCA
3. Use that PCA to transform the validation & test set

##### 4.1. For training set

In [ ]:
# keep 1100 components which contribute to > 90 %
pca_HOG_std_2 = PCA(n_components=1100)
pca_HOG_std_2.fit(HOG_list_std)
HOG_PCA_train = pca_HOG_std_2.transform(HOG_list_std)
print("Original shape:   ", HOG_list_std.shape)
print("Transformed shape:", HOG_PCA_train.shape)

In [ ]:
# HOG for train set --- standardization again
std_scale_train_2 = preprocessing.StandardScaler().fit(HOG_PCA_train)

In [ ]:
# Save in file
X_HOG_std_train = std_scale_train_2.transform(HOG_PCA_train)
X_HOG_train_dff = pd.DataFrame(data = X_HOG_std_train)
X_HOG_train_df = pd.DataFrame(data = dfFoldTraining_1["short_filenames"])

X_HOG_train_df = pd.concat([X_HOG_train_df,X_HOG_train_dff], axis=1)
X_HOG_train_df.columns = pd.RangeIndex(X_HOG_train_df.columns.size)

display(X_HOG_train_df.head(5), X_HOG_train_df.shape)

X_HOG_train_df.to_csv("..//_inputs//_image_features//new//X-HOG_std_PCA_1100_std-train-fold_1.csv", header=False, index=False)

##### 4.2. For validation set

In [ ]:
#Extract HOG features for the validation set
HOG_validation_list = extract_hog(dfFoldValidation_1)

In [ ]:
# standard deviation normalization
HOG_validation_list_std = train_std_norm.transform(HOG_validation_list)

In [ ]:
# Transform the HOG features using above PCA fitting
HOG_PCA_validation = pca_HOG_std_2.transform(HOG_validation_list_std)

In [ ]:
print("Original shape:   ", HOG_validation_list.shape)
print("Transformed shape:", HOG_PCA_validation.shape)

In [ ]:
# standard deviation normalization using above std_scale_train = preprocessing.StandardScaler().fit(HOG_PCA_train)
X_HOG_std_validation = std_scale_train_2.transform(HOG_PCA_validation)

In [ ]:
X_HOG_train_dff = pd.DataFrame(data = X_HOG_std_validation)
X_HOG_train_df = pd.DataFrame(data = dfFoldValidation_1["short_filenames"])

X_HOG_train_df = pd.concat([X_HOG_train_df,X_HOG_train_dff], axis=1)
X_HOG_train_df.columns = pd.RangeIndex(X_HOG_train_df.columns.size)

display(X_HOG_train_df.head(5), X_HOG_train_df.shape)

X_HOG_train_df.to_csv("..//_inputs//_image_features//new//X-HOG_std_PCA_1100_std-validation-fold_1.csv", header=False, index=False)

##### 4.2. For test set

<u><b> Remarks :</b></u> We use 4-fold cross validaiton. Then, we need also to compute each kind of features for test set.
So, for the test set, we extract 4 sets of features for each fold

In [ ]:
dfTest = pd.read_csv('..//_inputs//_images_Zooscan//ZooScan-test_img.csv')

In [ ]:
#Extract HOG features for the test set
HOG_test_list = extract_hog(dfTest)

In [ ]:
# standard deviation normalization 
HOG_test_list_std = train_std_norm.transform(HOG_test_list)
# Transform the HOG features using above PCA fitting
HOG_PCA_test = pca_HOG_std_2.transform(HOG_test_list_std)

In [ ]:
print("Original shape:   ", HOG_test_list.shape)
print("Transformed shape:", HOG_PCA_test.shape)

In [ ]:
# standard deviation normalization using above std_scale_train = preprocessing.StandardScaler().fit(HOG_PCA_train)
X_HOG_std_test = std_scale_train_2.transform(HOG_PCA_test)

In [ ]:
X_HOG_train_dff = pd.DataFrame(data = X_HOG_std_test)
X_HOG_train_df = pd.DataFrame(data = dfTest["short_filenames"])

X_HOG_train_df = pd.concat([X_HOG_train_df,X_HOG_train_dff], axis=1)
X_HOG_train_df.columns = pd.RangeIndex(X_HOG_train_df.columns.size)

display(X_HOG_train_df.head(5), X_HOG_train_df.shape)

X_HOG_train_df.to_csv("..//_inputs//_image_features//new//X-HOG_std_PCA_1100_std-test-fold_1.csv", header=False, index=False)

### 4.2.2. For fold 2
#### 1. Read path of fold 2

In [ ]:
dfFoldTraining_1 = pd.read_csv('..//_inputs//_images_Zooscan//_Zooscan-training-fold_2.csv')
dfFoldValidation_1 = pd.read_csv('..//_inputs//_images_Zooscan//_Zooscan-validation-fold_2.csv')

#### 2. Extracting HOG feature for the training set

In [ ]:
HOG_list = extract_hog(dfFoldTraining_1)

In [ ]:
# create a standard deviation normalization for later uses
train_std_norm = StandardScaler().fit(HOG_list)

In [ ]:
# Standard deviation normalization
HOG_list_std = train_std_norm.transform(HOG_list)

In [ ]:
display(HOG_list_std)

In [ ]:
print('Total HOG features:',(HOG_list_std.shape))

#### 3. PCA analysis on the training set

In [ ]:
pca_HOG_std = PCA().fit(HOG_list_std)

##### 3.1. Plot PCA components and CEV

From this, we can know number of components to keep

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5), tight_layout=True)

ax.plot(np.cumsum(pca_HOG_std.explained_variance_ratio_)*100, linewidth=2)
ax.grid(color='r', linestyle='--', linewidth=1)

ax.set_xlabel('Number of components')
ax.set_ylabel('Cumulative explained variance');

ax.set_yticks(np.arange(0,105,5))
ax.set_xticks(np.arange(0,HOG_list.shape[1],100))

ax.axhline(y=90, linewidth=3, color='g', alpha=0.5)

# ax.plot(800, 91, marker="o", markersize=10, markeredgecolor="red", markerfacecolor="green")

ax.set_title("PCA analysis on HOG features of training set")

In [ ]:
(np.cumsum(pca_HOG_std.explained_variance_ratio_)*100)[1100]

<b>Remarks: More than 90% of variance is explained by first 1100 components</b>

#### 4. Fit PCA to the HOG features

1. We fit PCA (with n_components to keep) onto the training set
2. Transform the training set with that PCA
3. Use that PCA to transform the validation & test set

##### 4.1. For training set

In [ ]:
# keep 1100 components which contribute to > 90 %
pca_HOG_std_2 = PCA(n_components=1100)
pca_HOG_std_2.fit(HOG_list_std)
HOG_PCA_train = pca_HOG_std_2.transform(HOG_list_std)
print("Original shape:   ", HOG_list_std.shape)
print("Transformed shape:", HOG_PCA_train.shape)

In [ ]:
# HOG for train set --- standardization again
std_scale_train_2 = preprocessing.StandardScaler().fit(HOG_PCA_train)

In [ ]:
# Save in file
X_HOG_std_train = std_scale_train_2.transform(HOG_PCA_train)
X_HOG_train_dff = pd.DataFrame(data = X_HOG_std_train)
X_HOG_train_df = pd.DataFrame(data = dfFoldTraining_1["short_filenames"])

X_HOG_train_df = pd.concat([X_HOG_train_df,X_HOG_train_dff], axis=1)
X_HOG_train_df.columns = pd.RangeIndex(X_HOG_train_df.columns.size)

display(X_HOG_train_df.head(5), X_HOG_train_df.shape)

X_HOG_train_df.to_csv("..//_inputs//_image_features//new//X-HOG_std_PCA_1100_std-train-fold_2.csv", header=False, index=False)

##### 4.2. For validation set

In [ ]:
#Extract HOG features for the validation set
HOG_validation_list = extract_hog(dfFoldValidation_1)

In [ ]:
# standard deviation normalization
HOG_validation_list_std = train_std_norm.transform(HOG_validation_list)

In [ ]:
# Transform the HOG features using above PCA fitting
HOG_PCA_validation = pca_HOG_std_2.transform(HOG_validation_list_std)

In [ ]:
print("Original shape:   ", HOG_validation_list.shape)
print("Transformed shape:", HOG_PCA_validation.shape)

In [ ]:
# standard deviation normalization using above std_scale_train = preprocessing.StandardScaler().fit(HOG_PCA_train)
X_HOG_std_validation = std_scale_train_2.transform(HOG_PCA_validation)

In [ ]:
X_HOG_train_dff = pd.DataFrame(data = X_HOG_std_validation)
X_HOG_train_df = pd.DataFrame(data = dfFoldValidation_1["short_filenames"])

X_HOG_train_df = pd.concat([X_HOG_train_df,X_HOG_train_dff], axis=1)
X_HOG_train_df.columns = pd.RangeIndex(X_HOG_train_df.columns.size)

display(X_HOG_train_df.head(5), X_HOG_train_df.shape)

X_HOG_train_df.to_csv("..//_inputs//_image_features//new//X-HOG_std_PCA_1100_std-validation-fold_2.csv", header=False, index=False)

##### 4.2. For test set

<u><b> Remarks :</b></u> We use 4-fold cross validaiton. Then, we need also to compute each kind of features for test set.
So, for the test set, we extract 4 sets of features for each fold

In [ ]:
dfTest = pd.read_csv('..//_inputs//_images_Zooscan//ZooScan-test_img.csv')

In [ ]:
#Extract HOG features for the test set
HOG_test_list = extract_hog(dfTest)

In [ ]:
# standard deviation normalization 
HOG_test_list_std = train_std_norm.transform(HOG_test_list)
# Transform the HOG features using above PCA fitting
HOG_PCA_test = pca_HOG_std_2.transform(HOG_test_list_std)

In [ ]:
print("Original shape:   ", HOG_test_list.shape)
print("Transformed shape:", HOG_PCA_test.shape)

In [ ]:
# standard deviation normalization using above std_scale_train = preprocessing.StandardScaler().fit(HOG_PCA_train)
X_HOG_std_test = std_scale_train_2.transform(HOG_PCA_test)

In [ ]:
X_HOG_train_dff = pd.DataFrame(data = X_HOG_std_test)
X_HOG_train_df = pd.DataFrame(data = dfTest["short_filenames"])

X_HOG_train_df = pd.concat([X_HOG_train_df,X_HOG_train_dff], axis=1)
X_HOG_train_df.columns = pd.RangeIndex(X_HOG_train_df.columns.size)

display(X_HOG_train_df.head(5), X_HOG_train_df.shape)

X_HOG_train_df.to_csv("..//_inputs//_image_features//new//X-HOG_std_PCA_1100_std-test-fold_2.csv", header=False, index=False)